# Introduction



---



---



In this notebook, we will implement the following tests for the NONLINEAR KOPTEVA PROBLEM:

1) For each initial guess in my "initial guess pool" for the Kopteva problem


a) For each epsilon:

- Gauss Seidal MP-Iteration (Fieldsplit Code). Compute Hessian Mesh Density Function based on the current physical solution and mesh, take $\alpha_{hat} = \epsilon^{-3/4}$ (for $m=0$) or $\alpha_{hat} = \epsilon^{-5/4}$ (for $m=1$) for the scaling on the Hessian in the Mesh Densitu Function M.

- Continuation in $u$ and $x$

b) Compare errors, solutions, etc


# Results and Comments


---



---


- Meshes and Physical Solutions look good at small epsilons.

- Need more or better initial guesses. I do not yet have initial guesses that allow me to compute all the solutions I have seen.

- MP-Iteration stopped with a mesh_tol = (1/10) * epsilon



In [1]:
# install firedrake

# hide output
%%capture

try:
    from firedrake import *
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    from firedrake import *

In [2]:
# Code in this cell makes plots appear an appropriate size and resolution in the browser window

%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (11, 6)

In [3]:
# import firedrake tools

from firedrake import *
import numpy as np
import matplotlib.pyplot as plt # firedrake makes use of matplotlib tools
from firedrake.pyplot import tripcolor, tricontour, triplot #firedrake plotting
from IPython.display import display
from mpl_toolkits import mplot3d



# Setup

In [4]:
# SETUP

# Mesh and space
N = 100
xi_mesh = UnitIntervalMesh(N)
V_mesh = FunctionSpace(xi_mesh, 'CG', 1)
V_phys = FunctionSpace(xi_mesh, "CG", 4)
x = Function(V_mesh).interpolate(SpatialCoordinate(xi_mesh)[0])
perm_x =  np.argsort(x.dat.data[:])



# Solver paramaters


newton_params = {
        'snes_type': 'newtonls',
        'snes_monitor': None,
        "snes_converged_reason": None,
        'ksp_type': 'preonly',
        'snes_linesearch_type': 'l2',
        # 'snes_linesearch_monitor': None,
        'snes_linesearch_damping': 0.5,
        'pc_type' : 'lu',
        'snes_rtol': 1e-8,
        'snes_stol': 1e-8,
        'snes_max_it': 100,
}


lu_params = {
    'snes_type': 'ksponly',
    # 'snes_monitor': None,
    'mat_type': 'aij',
    'ksp_type': 'preonly',
    'pc_type': 'lu',
}


mesh_params = {
    'snes_type': 'ksponly',
    'snes_monitor': None,
    'mat_type': 'aij',
    'ksp_type': 'richardson',
    'ksp_richardson_scale': 0.5,
    'pc_type': 'lu',
}



GS_params = {
    'mat_type': 'nest',
    'snes_type': 'nrichardson',
    'snes_monitor': None,
    'npc_snes_type': 'python',
    'npc_snes_python_type': __name__+'.GaussSeidelSNES',
    'snes_linesearch_type': 'basic',
    'snes_linesearch_monitor': None,
    'snes_linesearch_damping': 0.5,
    'npc_gs_0': lu_params,
    'npc_gs_1': lu_params,
    'npc_gs_2': lu_params,
    'npc_gs_3': newton_params,
    'snes_max_it': 20,
    'snes_rtol': 1e-12,
    'snes_atol': 1e-50,
    'snes_stol': 1e-12,
}




FS_params = {
    'mat_type': 'nest',
    'snes_type': 'nrichardson',
    'snes_monitor': None,
    'snes_converged_reason': None,
    'npc_snes_type': 'python',
    'npc_snes_python_type': __name__+'.FieldsplitSNES',
    'npc_snes_fieldsplit_type': "multiplicative",
    'snes_linesearch_type': 'basic',
    # 'snes_linesearch_monitor': None,
    'snes_linesearch_damping': 1/2,
    'npc_snes_fieldsplit_0': mesh_params,
    'npc_snes_fieldsplit_1': newton_params,
    'npc_snes_fieldsplit_2': lu_params,
    'snes_max_it': 100,
    'snes_rtol': 1e-8,
    'snes_atol': 1e-50,
    'snes_stol': 1e-8,
}

# The Solvers

In [5]:
# Class for the MP solver.
from firedrake.dmhooks import get_appctx, get_function_space


class GaussSeidelSNES(SNESBase):
    prefix = "gs_"

    def initialize(self, snes):
        ctx = get_appctx(snes.dm)
        W = get_function_space(snes.dm)
        self.sol = ctx._problem.u_restrict
        split_ctxs = ctx.split([(i,) for i in range(len(W))])
        full_prefix = snes.getOptionsPrefix() + self.prefix
        for i, ctx in enumerate(split_ctxs):
            NonlinearVariationalSolver(
                ctx._problem, options_prefix=full_prefix+str(i))
        self.solvers = tuple(
            NonlinearVariationalSolver(
                ctx._problem, options_prefix=full_prefix+str(i))
            for i, ctx in enumerate(split_ctxs)
        )

    def update(self, snes):
        pass

    def step(self, snes, x, f, y):
        with self.sol.dat.vec_wo as vec:
            x.copy(vec)

        solve_count = 1

        for solver, u in zip(self.solvers, self.sol.subfunctions):

            print('Solver Counter = ', solve_count)
            print()
            print()
            solve_count += 1


            solver._problem.u_restrict.assign(u)
            solver.solve()
            u.assign(solver._problem.u_restrict)




In [6]:
from firedrake.preconditioners.base import SNESBase
from firedrake.petsc import PETSc
from firedrake.dmhooks import get_appctx, get_function_space
from firedrake.function import Function



__all__ = ("FieldsplitSNES",)


class FieldsplitSNES(SNESBase):
    prefix = "fieldsplit_"

    # TODO:
    #   - Allow setting field grouping/ordering like fieldsplit

    @PETSc.Log.EventDecorator()
    def initialize(self, snes):
        from firedrake.variational_solver import NonlinearVariationalSolver  # ImportError if we do this at file level
        ctx = get_appctx(snes.dm)
        W = get_function_space(snes.dm)
        self.sol = ctx._problem.u_restrict

        # buffer to save solution to outer problem during solve
        self.sol_outer = Function(self.sol.function_space())

        # buffers for shuffling solutions during solve
        self.sol_current = Function(self.sol.function_space())
        self.sol_new = Function(self.sol.function_space())

        # options for setting up the fieldsplit
        snes_prefix = snes.getOptionsPrefix() + 'snes_' + self.prefix
        # options for each field
        sub_prefix = snes.getOptionsPrefix() + self.prefix

        snes_options = PETSc.Options(snes_prefix)
        self.fieldsplit_type = snes_options.getString('type', 'additive')
        if self.fieldsplit_type not in ('additive', 'multiplicative'):
            raise ValueError(
                'FieldsplitSNES option snes_fieldsplit_type must be'
                ' "additive" or "multiplicative"')

        split_ctxs = ctx.split([(i,) for i in range(len(W))])

        self.solvers = tuple(
            NonlinearVariationalSolver(
                ctx._problem, appctx=ctx.appctx,
                options_prefix=sub_prefix+str(i))
            for i, ctx in enumerate(split_ctxs)
        )

    def update(self, snes):
        pass

    @PETSc.Log.EventDecorator()
    def step(self, snes, x, f, y):
        # store current value of outer solution
        self.sol_outer.assign(self.sol)

        # the full form in ctx now has the most up to date solution
        with self.sol_current.dat.vec_wo as vec:
            x.copy(vec)
        self.sol.assign(self.sol_current)

        # The current snes solution x is held in sol_current, and we
        # will place the new solution in sol_new.
        # The solvers evaluate forms containing sol, so for each
        # splitting type sol needs to hold:
        #   - additive: all fields need to hold sol_current values
        #   - multiplicative: fields need to hold sol_current before
        #       they are are solved for, and keep the updated sol_new
        #       values afterwards.


        # solver_count = 0
        # solver_list = ["mesh solve", "physical solve", "mesh update", "alpha solve"]
        # print()
        # print()
        # print()
        # print()
        # print("MP ITERATION")
        # print('_'*50)
        # print()


        for solver, u, ucurr, unew in zip(self.solvers,
                                          self.sol.subfunctions,
                                          self.sol_current.subfunctions,
                                          self.sol_new.subfunctions):
            # print(solver_list[solver_count])
            # solver_count += 1
            # print()

            solver.solve()
            unew.assign(u)
            if self.fieldsplit_type == 'additive':
                u.assign(ucurr)



        # # plot our current solutions
        # x_curr = self.sol.subfunctions[0]
        # perm_curr = np.argsort(x_curr.dat.data[:])
        # plt.plot(x_curr.dat.data[:], np.zeros_like(x_curr.dat.data[:]), marker = "|")
        # plt.title("Current Mesh")
        # plt.xlabel("x mesh")
        # plt.ylabel("0")
        # plt.show()
        # print()
        # print()


        with self.sol_new.dat.vec_ro as vec:
            vec.copy(y)
            y.aypx(-1, x)

        # restore outer solution
        self.sol.assign(self.sol_outer)

# Set Initial Guesses

In [7]:


def setting_guesses(n):


    # set shishkin mesh
    beta = 0.99/np.sqrt(2)
    tau_a = 1/2
    tau_b = 2*epsilon*(1/beta)*np.log(N)
    tau = min(tau_a, tau_b)
    h = tau/(N/4.0)
    x_s = np.zeros(N+1)
    for i in range(1,int(N/4)+1):
      x_s[i] = i*h
    for i in range(int(N/4)+1,int(N/2)+1):
      x_s[i] = tau+(0.5-tau)*(i-N/4)/(N/4)
    for i in range(int(N/2)+1,N+1):
      x_s[i] = 1-x_s[N-i]


    if n == 0: # u = 0

        u.interpolate(0)
        # set x guess as uniform
        x.interpolate(SpatialCoordinate(xi_mesh)[0])

    elif n == 1: # u = 0

        u.interpolate(0)
        # set x guess as shishkin
        x.dat.data[perm_x] = x_s[:]



    elif n == 2: # u = 0
        u.interpolate(1)
        # set x guess as uniform
        x.interpolate(SpatialCoordinate(xi_mesh)[0])



    elif n == 3: # u = 0
        u.interpolate(1)
        # set x guess as shishkin
        x.dat.data[perm_x] = x_s[:]



    elif n == 4: # u = xi
        u.interpolate(SpatialCoordinate(xi_mesh)[0])
        # set x guess as uniform
        x.interpolate(SpatialCoordinate(xi_mesh)[0])



    elif n == 5: # u = xi
        u.interpolate(SpatialCoordinate(xi_mesh)[0])
        # set x guess as shishkin
        x.dat.data[perm_x] = x_s[:]



    elif n == 6: # u = xi + 1/2
        u.interpolate(SpatialCoordinate(xi_mesh)[0] + 1/2)
        # set x guess as uniform
        x.interpolate(SpatialCoordinate(xi_mesh)[0])



    elif n == 7: # u = xi + 1/2
        u.interpolate(SpatialCoordinate(xi_mesh)[0] + 1/2)
        # set x guess as shishkin
        x.dat.data[perm_x] = x_s[:]



    elif n == 8: # u = xi + 3/2
        u.interpolate(SpatialCoordinate(xi_mesh)[0] + 3/2)
        # set x guess as uniform
        x.interpolate(SpatialCoordinate(xi_mesh)[0])



    elif n == 9: # u = xi + 3/2
        u.interpolate(SpatialCoordinate(xi_mesh)[0] + 3/2)
        # set x guess as shishkin
        x.dat.data[perm_x] = x_s[:]




    elif n == 10: # u = x
        # set x guess as uniform
        x.interpolate(SpatialCoordinate(xi_mesh)[0])
        u.interpolate(x)



    elif n == 11: # u = x
        # set x guess as shishkin
        x.dat.data[perm_x] = x_s[:]
        u.interpolate(x)



    elif n == 12: # u = x + 1/2
        # set x guess as uniform
        x.interpolate(SpatialCoordinate(xi_mesh)[0])
        u.interpolate(x + 1/2)



    elif n == 13: # u = x + 1/2
        # set x guess as shishkin
        x.dat.data[perm_x] = x_s[:]
        u.interpolate(x + 1/2)




    elif n == 14: # u = x + 3/2
        # set x guess as uniform
        x.interpolate(SpatialCoordinate(xi_mesh)[0])
        u.interpolate(x + 3/2)



    elif n == 15: # u = x + 3/2
        # set x guess as shishkin
        x.dat.data[perm_x] = x_s[:]
        u.interpolate(x + 3/2)

    elif n == 16:
      x.interpolate(SpatialCoordinate(xi_mesh)[0])

      len_u = len(u.dat.data[:])
      half_len = len_u//2

      print(len_u, half_len)
      print()

      u.dat.data[:half_len] = 1/2
      u.dat.data[-half_len:] = 3/2


    else:
      print('n = ', n)
      print('Invalid Input')
      print()


# Numerical Test - Looping Over Guesses

In [8]:

for n in range(17):

    print()
    print('*'*100)
    print()
    print("INITIAL GUESS NNUMBER", n)
    print()
    print('*'*100)
    print()

    # SET EPSILONS
    epsilon = 0.1
    eps_final = 0.01
    eps_change = 9/10
    results = []



    # M PARAMETERS (m=0 IS L2)
    p = 2
    q = 2
    m = 0
    exp_M = Constant( (2*q) / (1 + q*(2-m)) )
    alpha_power = -3/4
    alpha_scale = 5


    # INITIAL GUESSES
    x = Function(V_mesh)
    u = Function(V_phys)
    setting_guesses(n)


    x_save = Function(V_mesh)
    u_save = Function(V_phys)
    results = []


    # EPSILON CONTINUATION
    ##########################################################


    while epsilon >= eps_final:


        print()
        print()
        print('-_'*100)
        print('EPSILON = ', epsilon)
        print('-_'*100)
        print()
        print()


        # RESET SOLVER PARAMETERS

        FS_params = {
        'mat_type': 'nest',
        'snes_type': 'nrichardson',
        'snes_monitor': None,
        'snes_converged_reason': None,
        'npc_snes_type': 'python',
        'npc_snes_python_type': __name__+'.FieldsplitSNES',
        'npc_snes_fieldsplit_type': "multiplicative",
        'snes_linesearch_type': 'basic',
        # 'snes_linesearch_monitor': None,
        'snes_linesearch_damping': 1/2,
        'npc_snes_fieldsplit_0': mesh_params,
        'npc_snes_fieldsplit_1': newton_params,
        'npc_snes_fieldsplit_2': lu_params,
        'snes_max_it': 100,
        'snes_rtol': (1/10)*epsilon,
        'snes_atol': 1e-50,
        'snes_stol': (1/10)*epsilon,
        }


        # STARTING PHYSICAL SOLVE AT NEW EPSILON ON OLD MESH

        # Test and Solution Functions
        u_new = Function(V_phys)
        v = TestFunction(V_phys)

        # Jacobian of x
        Jx = x.dx(0) #dx/dxi

        # Derivatives
        u_deriv = (1/Jx) * u_new.dx(0)
        v_deriv = (1/Jx) * v.dx(0)

        # The residual
        F1 = ( (epsilon**2) * u_deriv * v_deriv) * Jx * dx
        F2 = (u_new * (u_new-1) * (u_new - x - 3/2) * v) * Jx * dx
        F = F1 + F2

        # BC's
        bcs = [DirichletBC(V_phys, 3/2, 1), DirichletBC(V_phys, 1/2, 2)]


        # Solve
        try:
          u_new.interpolate(u)
          solve( F==0, u_new, bcs = bcs, solver_parameters = newton_params)
        except Exception as e:
          print('The physical solve failed due to the following message:')
          print(e)
          print()
          break

        # Update u
        u.dat.data[:] = u_new.dat.data[:]


        # plot
        plt.plot(x.dat.data[perm_x], Function(V_mesh).interpolate(u).dat.data[perm_x], label = 'u')
        plt.plot(x.dat.data[perm_x], [0 for i in range(len(x.dat.data[perm_x]))], marker = '|', label = 'x mesh')
        plt.xlabel('x')
        plt.ylabel('u')
        plt.title('Starting Physical Solve at Epsilon = ' + str(epsilon))
        plt.legend()
        plt.show()
        print()
        print()



        # MP ITERATION
        ################################################


        # SETUP
        W = V_mesh * V_phys * V_mesh
        xux = Function(W)
        yvy = TestFunction(W)

        # set our initial guesses
        xux.sub(0).interpolate(x)
        xux.sub(1).interpolate(u)
        xux.sub(2).interpolate(x)


        # Retrieve our functions
        x, u, x_old = split(xux)
        y,v, y_old = split(yvy)


        # MESH DENSITY FUNCTION BASED ON XOLD

        # x_old jacobian
        jacobx = x_old.dx(0) #dx/dxi

        # compute the derivatives of our u
        u_pr_lin = u.dx(0)
        u_prpr_lin = u_pr_lin.dx(0)
        x_prpr_lin = jacobx.dx(0)
        du_dx_lin = (1/jacobx) * u_pr_lin
        du2_dx2_lin = ( u_prpr_lin -  du_dx_lin * x_prpr_lin) * (( 1/jacobx )**2)

        # compute the Hessian M
        alpha = alpha_scale * epsilon ** (alpha_power)
        inside_bit_lin = 1 + (1/alpha) * abs(du2_dx2_lin)
        M_lin = inside_bit_lin ** (exp_M)


        # MESH PROBLEM

        # Jacobian
        Jm = x.dx(0)
        # y' where y is test for x
        y_pr_term = y.dx(0)
        # Residual
        F_mesh = (M_lin * Jm * y_pr_term)*dx


        # PHYSICAL SOLVE BASED ON SOLUTION FOR X


        # Physical PDE

        u_deriv = (1/Jm) * u.dx(0)
        v_deriv = (1/Jm) * v.dx(0)
        # The residual
        F1 = ( (epsilon**2) * u_deriv * v_deriv) * Jm * dx
        F2 = (u * (u-1) * (u - x - 3/2) * v) * Jm * dx
        F_phys = F1 + F2




        # THE TOTAL FORM

        F = F_mesh + F_phys + (x-x_old)*y_old*dx

        # bc's
        bcx0 = DirichletBC(W.sub(0), Constant(0), 1)
        bcx1 = DirichletBC(W.sub(0), Constant(1), 2)
        bcu0 = DirichletBC(W.sub(1), Constant(3/2), 1)
        bcu1 = DirichletBC(W.sub(1), Constant(1/2), 2)
        bcs = [bcx0, bcx1, bcu0, bcu1]


        # SOLVE

        try:
          NLVP = NonlinearVariationalProblem(F, xux, bcs=bcs)
          NLVS = NonlinearVariationalSolver(NLVP, solver_parameters=FS_params)
          NLVS.solve()
        except Exception as e:
          print()
          print()
          print('Solver failed because of the following message:')
          print(e)
          print()


        # L2 AND ENERGY NORM FOR RESULT OF CURRENT EPSILON


        print()
        print('_'*100)
        print('RESULTS')
        print('_'*100)
        print()

        # Pull our results
        x, u, x_old = xux.subfunctions




        # SAVE AND PLOT THE RESULTS FOR THE CURRENT EPSILON

        # save
        u_save.dat.data[:] = u.dat.data[:]
        x_save.dat.data[:] = x.dat.data[:]
        results.append([epsilon, u_save, x_save])

        print('PLOTS:')

        # plot
        plt.plot(x.dat.data[perm_x], Function(V_mesh).interpolate(u).dat.data[perm_x], label = 'u')
        plt.plot(x.dat.data[perm_x], [0 for i in range(len(x.dat.data[perm_x]))], marker = '|', label = 'x mesh')
        plt.xlabel('x')
        plt.ylabel('u')
        plt.title('The Results for Epsilon = ' + str(epsilon))
        plt.legend()
        plt.show()
        print()
        print()



        # UPDATE EPSILON
        epsilon = epsilon * eps_change




Output hidden; open in https://colab.research.google.com to view.